### Choropleth Map SMR

In [ ]:
# import the library
from shapely.geometry import shape
import geopandas as gpd

import folium, json, requests
import pandas as pd
import math

In [ ]:
df=pd.read_csv('../data/SMRs_all.csv', sep=';').reset_index(drop=True)
df.drop(['geo_point_2d'],axis=1,inplace=True)
df.head(1)

In [ ]:
#add 0 before the 4 digit codgeo
def fours(ge):
        k=len(ge)
        if k == 4:
            m= '0'+ge
            return m
        else:
            return ge
        
df['CODGEO']=df['CODGEO'].apply(fours)

In [ ]:
df[df['CODGEO']=='01073']

In [ ]:
df1=pd.read_csv('../data/communes_codego_definitiva.csv', index_col=0).reset_index(drop=True)
#df1.drop(['Commune'],axis=1,inplace=True)
df1.head()

In [ ]:
df1[df1['CODGEO']=='01073']

In [ ]:
maindf=pd.merge(df,df1, on='CODGEO')
maindf.head(2)

In [ ]:
maindf[maindf['CODGEO']=='75104']

In [ ]:
maindf['SMR'] = maindf['SMR'].str.replace(',', '.').astype(float)

In [ ]:
#maindf=maindf[maindf['SMR'] >= 1]
maindf=maindf[maindf['Population'] >= 0.1]
maindf1=maindf[['CODGEO','SMR','lat','lon','Commune','Population']]
maindf1.head(2)

##### polygons

In [ ]:
poly=pd.read_csv('../data/communes_polygon.csv', index_col=0)
poly.head(1)

In [ ]:
#Apply 4s to CODGEO        
poly['CODGEO']=poly['CODGEO'].apply(fours)
poly.head()

In [ ]:
poly = poly.rename(columns = {"CODGEO":"code"})

In [ ]:
#maindf1[maindf1['code'] == '01073']

In [ ]:
maindf1 = maindf1.rename(columns = {"CODGEO":"code"})

In [ ]:
final_df = poly.merge(maindf1, on = "code")
final_df=final_df.dropna()
final_df.head(2)

In [ ]:
#ADD new SMR colums
dfsmr=pd.read_csv('../data/simplified_SMRs_all.csv', sep=';', index_col=0)
#dfsmr.drop(['geo_point_2d'],axis=1,inplace=True)
dfsmr.head(1)

In [ ]:
#Apply 4s to CODGEO        
dfsmr['CODGEO']=dfsmr['CODGEO'].apply(fours)
dfsmr = dfsmr.rename(columns = {"CODGEO":"code"})
dfsmr.head(2)

In [ ]:
#MERGE new table
final_df = final_df.merge(dfsmr, on = "code")
final_df.head(2)

In [ ]:
final_df = final_df.rename(columns = {"geo_shape1":"geometry"})

In [ ]:
final_df['cityIndicator'].describe()

In [ ]:
from shapely import wkt


In [ ]:
final_df.crs = 'epsg:4326'

In [ ]:
final_df['geometry'] = final_df['geometry'].apply(wkt.loads)
final_df = gpd.GeoDataFrame(final_df, crs='epsg:4326')
type(final_df)

In [ ]:
#gdf=final_df[final_df['code'] == '63402']
#gdf.info()
#final_df=gdf.copy()
#final_df=final_df[23200:28200]
final_df.info()

In [ ]:
final_df['SMR'].describe()

##### build map

In [ ]:
# Make an empty map
m = folium.Map(location = [46.71109,1.7191036], tiles=None,zoom_start=6,min_zoom = 5,max_zoom = 10,control=False)


folium.TileLayer(
    tiles="cartodbpositron",zoom_start=6,min_zoom = 5,max_zoom = 10,
    name='darkmatter',
    control=False,
    opacity=0.7
).add_to(m)


##### choropleth Département

In [ ]:


#bins = list(final_df["dptIndicator"].quantile([0,0.19,0.22, 0.23,   0.38,0.4,0.4181,  1]))

bins = list(final_df["dptIndicator"].quantile([0,0.19,0.22, 0.23,   0.38,0.4,0.4181,  1]))


choropleth=folium.Choropleth(geo_data=final_df, name="dptIndicator",
                  data=final_df,
                  columns=['code',"dptIndicator"],
                  smooth_factor=0, line_opacity=0,
                  fill_color = 'RdYlGn_r',
                  fill_opacity=0.7,
                  bins=bins,
                             control=False,legend_name='Département Indicator',
                  key_on="feature.properties.code")

choropleth.add_to(m) 
#m

In [ ]:
#m.save('departement_indicator.html')

In [ ]:
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('departement_indicator.png')

##### choropleth Canton

In [ ]:
# Make an empty map
m = folium.Map(location = [46.71109,1.7191036], tiles=None,zoom_start=6,min_zoom = 5,max_zoom = 10,control=False)


folium.TileLayer(
    tiles="cartodbpositron",zoom_start=6,min_zoom = 5,max_zoom = 10,
    name='darkmatter',
    control=False,
    opacity=0.7
).add_to(m)


In [ ]:


#bins = list(final_df["cantonIndicator"].quantile([0,0.06,0.08,0.1,     0.8,0.91, 0.95,  1]))
bins = list(final_df["cantonIndicator"].quantile([0,0.04,    0.08,0.15,     0.6,0.7, 0.8,  1]))


choropleth=folium.Choropleth(geo_data=final_df, name="cantonIndicator",
                  data=final_df,
                  columns=['code',"cantonIndicator"],
                  smooth_factor=0, line_opacity=0,
                  fill_color = 'RdYlGn_r',
                  fill_opacity=0.7,
                  bins=bins,
                             control=False,legend_name='Canton Indicator',
                  key_on="feature.properties.code")

choropleth.add_to(m) 
#m

In [ ]:
#m.save('canton_indicator.html')

In [ ]:
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('canton_indicator.png')

##### choropleth City

In [ ]:
# Make an empty map
m = folium.Map(location = [46.71109,1.7191036], tiles=None,zoom_start=6,min_zoom = 5,max_zoom = 10,control=False)


folium.TileLayer(
    tiles="cartodbpositron",zoom_start=6,min_zoom = 5,max_zoom = 10,
    name='darkmatter',
    control=False,
    opacity=0.7
).add_to(m)


In [ ]:

#bins = list(final_df["SMR"].quantile([0,0.15, 0.3,0.4, 0.75, 0.85,1]))
#bins = list(final_df["cityIndicator"].quantile([0,0.05,0.25,0.6,0.8,0.95,0.99,1]))
bins = list(final_df["cityIndicator"].quantile([0,0.01,0.02,0.1,     0.9,0.91, 0.95,  1]))


choropleth=folium.Choropleth(geo_data=final_df, name="cityIndicator",
                  data=final_df,
                  columns=['code',"cityIndicator"],
                  smooth_factor=0, line_opacity=0,
                  fill_color = 'RdYlGn_r',
                  fill_opacity=0.7,
                  bins=bins,
                             control=False,legend_name='City Indicator',
                  key_on="feature.properties.code")

choropleth.add_to(m) 

#m

In [ ]:
#m.save('city_indicator.html')

In [ ]:
import io
from PIL import Image

img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))
img.save('city_indicator.png')